<a href="https://colab.research.google.com/github/nandodeomkar/BFD-using-ResNet50/blob/main/Full_Fledged_Try_at_BFD_using_ResNet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras
!pip install opencv-python
!pip install numpy
!pip install matplotlib
!pip install scikit-learn

In [ ]:
import cv2
import os

# Set the path to the directory containing the images
dir_path = '/content/drive/MyDrive/Dataset/training'

# Set the desired size for the images
img_size = (224, 224)

# Loop through the files in the directory
for filename in os.listdir(dir_path):
    # Check if the file is an image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
        img = cv2.imread(os.path.join(dir_path, filename))

        # Resize the image to the desired size
        resized_img = cv2.resize(img, img_size)

        # Save the resized image with the same filename and extension
        cv2.imwrite(os.path.join(dir_path, filename), resized_img)


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50

# Set the path to the directory containing the training images
train_dir = '/content/drive/MyDrive/training'

# Set the batch size for training
batch_size = 32

# Create an ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest', preprocessing_function=lambda x: x/255.0)

# Load the ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

# Add a global average pooling layer and a dense output layer for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)

# Create the modified model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the weights of the base ResNet50 model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Create a generator for the training data
train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')

# Train the model for a few epochs
model.fit(train_generator, epochs=10)

# Save the trained model to Google Drive
model.save('/content/drive/My Drive/bone_fracture_detection.h5')


94765736/94765736 [==============================] - 3s 0us/step


Found 8873 images belonging to 2 classes.
Epoch 1/10
 71/278 [======>.......................] - ETA: 34:21 - loss: 0.7104 - accuracy: 0.5031

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input

# Set the path to the trained model file
model_file = '/content/drive/My Drive/bone_fracture_detection.h5'

# Load the trained model
model = load_model(model_file)

# Set the path to the input image
input_image = '/content/drive/MyDrive/Dataset/testing/fractured/image.jpg'

# Set the desired size for the image
img_size = (224, 224)

# Load the image and preprocess it
img = load_img(input_image, target_size=img_size)
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Use the model to predict the class label
pred = model.predict(x)[0]
if pred < 0.5:
    label = 'Not Fractured'
else:
    label = 'Fractured'

# Print the predicted label and the image filename
print('{} - {}'.format(input_image, label))
